In [ ]:
import pandas as pd
data_main=pd.read_csv("data.csv")


import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

# Set multiple gpus
strategy = tf.distribute.MirroredStrategy()


#change of the rating classes to new classes
import numpy as np

lbl_first=data_main["RATING"].values
lbl_final=lbl_first.copy()

dict_trans_calss={1:[1,2,3,4,5],
                 10:[10,11,12],
                  13:[13,14,15],
                  16:[16,17,18,19,20,21,22]
                  }


for calss_main_idx in dict_trans_calss:
  lst_class=dict_trans_calss[calss_main_idx]
  for idx, item in enumerate(lst_class):
    lbl_final = np.where(lbl_first == item, calss_main_idx, lbl_final)
    
lbl_final_rest=lbl_final.copy()
uni_class=np.unique(lbl_final)
for idx,item in enumerate(uni_class):
    lbl_final_rest=np.where(lbl_final == item, idx, lbl_final_rest)
unique, counts = np.unique(lbl_final_rest, return_counts=True)
print(np.asarray((unique, counts)).T)
data_pre=data_main.copy()
data_token=data_pre["cleantext"].values
##########################Preprocessing

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('words')

import numpy as np
import pandas as pd
import os
import re
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from cleantext import clean

# get lemmatized review
def get_lemmatized_text(corpus):
    lemmatizer = WordNetLemmatizer()
    return [' '.join([lemmatizer.lemmatize(word) for word in review.split()]) for review in corpus]
# get stemmed review
def get_stemmed_text(corpus):  
    stemmer = PorterStemmer()
    out_text=[' '.join([stemmer.stem(word) for word in review.split()]) for review in corpus]
    return out_text


# remove all stopwords in english review
def remove_stop_words(corpus):
    english_stop_words = stopwords.words('english')
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in english_stop_words])
        )
    return removed_stop_words

# preprocess review
def preprocess_txt(reviews):
    REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
    REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
    NO_SPACE = ""
    SPACE = " "
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, reviews.lower())]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    return reviews
# clean review by cleantext libaray
def clean_text(sent):
  clean_sent=clean(sent,
      fix_unicode=True,               # fix various unicode errors
      to_ascii=True,                  # transliterate to closest ASCII representation
      lower=True,                     # lowercase text
      no_line_breaks=False,           # fully strip line breaks as opposed to only normalizing them
      no_urls=True,                  # replace all URLs with a special token
      no_emails=True,                # replace all email addresses with a special token
      no_phone_numbers=True,         # replace all phone numbers with a special token
      no_numbers=False,               # replace all numbers with a special token
      no_digits=True,                # replace all digits with a special token
      no_currency_symbols=True,      # replace all currency symbols with a special token
      no_punct=True,                 # remove punctuations
      replace_with_punct="",          # instead of removing punctuations you may replace them
      replace_with_url="",
      replace_with_email="",
      replace_with_phone_number="",
      replace_with_number="",
      replace_with_digit="0",
      replace_with_currency_symbol="",
      lang="en" )
  return clean_sent

data_pre["text"]=data_pre["text"].apply(lambda x:clean_text(x))
data_pre["text"]=data_pre["text"].apply(lambda x:preprocess_txt(x))
data_pre["text"]=data_pre["text"].apply(lambda x:remove_stop_words(x))
data_pre["text"]=data_pre["text"].apply(lambda x:get_stemmed_text(x))

#############################Tokenization 

import matplotlib.pyplot as plt
text = data_token
data_token_re=data_token.copy()
from keras.preprocessing.text import Tokenizer
MAX_NB_WORDS=500000
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(data_token_re)
data_seq = tokenizer.texts_to_sequences(data_token_re)
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
from tensorflow.keras.preprocessing.sequence import pad_sequences
maxlen = 5000
data_seq_pad = pad_sequences(data_seq, padding='pre', maxlen=maxlen)


###############################Numeric part
from tensorflow.keras.utils import  to_categorical
lbl_binary=to_categorical(lbl_final_rest).astype(int)
# 4:101:mareket, 102:110:bond info, 111:157:finRatio
d1=dum
d2=data[list(data.iloc[:,4:101])]
d3=data[list(data.iloc[:,102:110])]
d4=data[list(data.iloc[:,111:157])]



from sklearn.preprocessing import MinMaxScaler
data_meta_normal1=d1.copy()
data_meta_normal2=d2.copy()
data_meta_normal3=d3.copy()
data_meta_normal4=d4.copy()


sce=MinMaxScaler()
for item_col in data_meta_normal2.columns:
    data_meta_normal2[item_col]=sce.fit_transform(d2[item_col].values.reshape(-1,1))

for item_col in data_meta_normal3.columns:
    data_meta_normal3[item_col]=sce.fit_transform(d3[item_col].values.reshape(-1,1))
    
for item_col in data_meta_normal4.columns:
    data_meta_normal4[item_col]=sce.fit_transform(d4[item_col].values.reshape(-1,1))
    
 


from tensorflow.keras.utils import to_categorical
lbl_binary=to_categorical(lbl_final_rest).astype(int)


# In[96]:


from sklearn.model_selection import train_test_split
# train split mode
trian_Percentage=0.80;

x_train, x_test, lbl_train, lbl_test=train_test_split(data_seq_pad, 
                                                      lbl_binary,
                                                      test_size=1-trian_Percentage,random_state=0)
x_train_txt, x_test_txt, lbl_train, lbl_test=train_test_split(data_token, 
                                                      lbl_binary,
                                                      test_size=1-trian_Percentage,random_state=0)
x_train_meta1, x_test_meta1, lbl_train, lbl_test=train_test_split(data_meta_normal1.values, 
                                                      lbl_binary,
                                                      test_size=1-trian_Percentage,random_state=0)
x_train_meta2, x_test_meta2, lbl_train, lbl_test=train_test_split(data_meta_normal2.values, 
                                                      lbl_binary,
                                                      test_size=1-trian_Percentage,random_state=0)
x_train_meta3, x_test_meta3, lbl_train, lbl_test=train_test_split(data_meta_normal3.values, 
                                                      lbl_binary,
                                                      test_size=1-trian_Percentage,random_state=0)
x_train_meta4, x_test_meta4, lbl_train, lbl_test=train_test_split(data_meta_normal4.values, 
                                                      lbl_binary,
                                                      test_size=1-trian_Percentage,random_state=0)




x_train_meta_re1=x_train_meta1.reshape(x_train_meta1.shape[0],x_train_meta1.shape[1],1)
x_test_meta_re1=x_test_meta1.reshape(x_test_meta1.shape[0],x_test_meta1.shape[1],1)

x_train_meta_re2=x_train_meta2.reshape(x_train_meta2.shape[0],x_train_meta2.shape[1],1)
x_test_meta_re2=x_test_meta2.reshape(x_test_meta2.shape[0],x_test_meta2.shape[1],1)

x_train_meta_re3=x_train_meta3.reshape(x_train_meta3.shape[0],x_train_meta3.shape[1],1)
x_test_meta_re3=x_test_meta3.reshape(x_test_meta3.shape[0],x_test_meta3.shape[1],1)

x_train_meta_re4=x_train_meta4.reshape(x_train_meta4.shape[0],x_train_meta4.shape[1],1)
x_test_meta_re4=x_test_meta4.reshape(x_test_meta4.shape[0],x_test_meta4.shape[1],1)

embedding_dim=500
n_class=lbl_binary.shape[1]


# In[98]:


from sklearn.utils import class_weight
class_weights_val = class_weight.compute_class_weight(class_weight='balanced',
                                                 classes=np.unique(np.argmax(lbl_test,axis=1)),
                                                 y=np.argmax(lbl_test,axis=1))
class_weight=dict()
for idx,val in enumerate(class_weights_val):
    class_weight[idx]=val

class_weight


# In[99]:


from tensorflow import keras

METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.CategoricalAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve

]
##########################################CNN

from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
def Create_CNN(data,nb_words,maxlen,embedding_dim,data_meta1,data_meta2,data_meta3,data_meta4):


  # meta1 input1
  input_meta1=keras.layers.Input(shape=data_meta1.shape[1:],dtype='float64',name="input_meta1")  
  layer_meta1=keras.layers.Conv1D(filters=64,
                                kernel_size=2,
                                activation='relu',
                                padding="same",
                                strides=(1),
                              name="cov_meta1.1")(input_meta1)
  layer_meta1=keras.layers.MaxPool1D(pool_size=(2),name="meta1_maxpool1")(layer_meta1)
  layer_meta1=keras.layers.Conv1D(filters=64,
                                 kernel_size=2,
                                 activation='relu',
                                 padding="same",
                                 strides=(1),
                                name="cov_meta1.2")(layer_meta1)
  #layer_meta1=keras.layers.GlobalAveragePooling1D(name="avg_meta1")(layer_meta1)
    
  # meta2 input1
  input_meta2=keras.layers.Input(shape=data_meta2.shape[1:],dtype='float64',name="input_meta2")  
  layer_meta2=keras.layers.Conv1D(filters=64,
                                kernel_size=2,
                                activation='relu',
                                padding="same",
                                strides=(1),
                              name="cov_meta2.1")(input_meta2)
  layer_meta2=keras.layers.MaxPool1D(pool_size=(2),name="meta2_maxpool1")(layer_meta2)
  layer_meta2=keras.layers.Conv1D(filters=64,
                                 kernel_size=2,
                                 activation='relu',
                                 padding="same",
                                 strides=(1),
                                name="cov_meta2.2")(layer_meta2)
 # layer_meta2=keras.layers.GlobalAveragePooling1D(name="avg_meta2")(layer_meta2)   
    
  # meta3 input1
  input_meta3=keras.layers.Input(shape=data_meta3.shape[1:],dtype='float64',name="input_meta3")  
  layer_meta3=keras.layers.Conv1D(filters=64,
                                kernel_size=2,
                                activation='relu',
                                padding="same",
                                strides=(1),
                              name="cov_meta3.1")(input_meta3)
  layer_meta3=keras.layers.MaxPool1D(pool_size=(2),name="meta3_maxpool1")(layer_meta3)
  layer_meta3=keras.layers.Conv1D(filters=64,
                                 kernel_size=2,
                                 activation='relu',
                                 padding="same",
                                 strides=(1),
                                name="cov_meta3.2")(layer_meta3)
 # layer_meta3=keras.layers.GlobalAveragePooling1D(name="avg_meta3")(layer_meta3)
    
    
  # meta4 input1
  input_meta4=keras.layers.Input(shape=data_meta4.shape[1:],dtype='float64',name="input_meta4")  
  layer_meta4=keras.layers.Conv1D(filters=64,
                                kernel_size=2,
                                activation='relu',
                                padding="same",
                                strides=(1),
                              name="cov_meta4.1")(input_meta4)
  layer_meta4=keras.layers.MaxPool1D(pool_size=(2),name="meta4_maxpool1")(layer_meta4)
  layer_meta4=keras.layers.Conv1D(filters=64,
                                 kernel_size=2,
                                 activation='relu',
                                 padding="same",
                                 strides=(1),
                                name="cov_meta4.2")(layer_meta4)
  #layer_meta4=keras.layers.GlobalAveragePooling1D(name="avg_meta4")(layer_meta4)


  # txt input
  word_input_cnn=keras.layers.Input(shape=(maxlen,),dtype='float64',name="txt_input_cnn")  
  em_layer_cnn=keras.layers.Embedding(input_dim=vocab_size, 
                                  output_dim=embedding_dim,input_length=maxlen,
                                  name="Embedd_cnn")(word_input_cnn)
  layer_data=keras.layers.Conv1D(filters=64,
                                 kernel_size=2,
                                 activation='relu',
                                 padding="same",
                                 strides=(2),
                                name="cov_1")(em_layer_cnn)
  layer_data=keras.layers.MaxPool1D(pool_size=(2),name="data_maxpool0")(layer_data)

  layer_data =keras.layers.BatchNormalization()(layer_data) 
  layer_data =keras.layers.Dropout(0.1)(layer_data)

  layer_data=keras.layers.Conv1D(filters=64,
                                 kernel_size=2,
                                 activation='relu',
                                 padding="same",
                                name="cov_2")(layer_data)
  layer_data=keras.layers.MaxPool1D(pool_size=(2),name="data_maxpool1")(layer_data)

  layer_data =keras.layers.BatchNormalization()(layer_data)                              
  layer_data =keras.layers.Dropout(0.1)(layer_data)
  layer_data=keras.layers.Conv1D(filters=64,
                                 kernel_size=2,
                                 strides=(2),
                                 activation='relu',
                                 padding="same",
                                name="cov_4")(layer_data)
  #layer_data=keras.layers.GlobalAveragePooling1D(name="data_maxpool2")(layer_data)



  # hybrid
  layer_last1 =keras.layers.Concatenate(axis=1)([layer_meta1,layer_meta2,layer_meta3,layer_meta4]) 
  layer_last = MultiHeadAttention(num_heads=5, key_dim=5, dropout=0.5, name = "MHSA")(layer_last1,layer_data)
  layer_last=keras.layers.Flatten()(layer_last)

  
  layer_last =keras.layers.Dense(64,activation="relu",
                                 name="dens_2")(layer_last)
  layer_last =keras.layers.Dropout(0.5)(layer_last)
  output= keras.layers.Dense(n_class, activation='softmax', name='out')(layer_last)
  model = Model(inputs=[word_input_cnn,input_meta1,input_meta2,input_meta3,input_meta4],
                       outputs=output)
  opti=keras.optimizers.Adam(learning_rate=0.00001) #,clipnorm=0.1, decay=1e-3/200.0)
  model.compile(loss='categorical_crossentropy',
                       optimizer=opti,
                       metrics=[METRICS])
  return model


# In[101]:


CNN_embedding=Create_CNN(data=data_seq_pad,
                      nb_words=MAX_NB_WORDS,
                      maxlen=maxlen,
                      embedding_dim=embedding_dim,
                      data_meta1=x_train_meta_re1,
                      data_meta2=x_train_meta_re2,
                      data_meta3=x_train_meta_re3,
                      data_meta4=x_train_meta_re4)

hist_CNN_embedding=CNN_embedding.fit([x_train,x_train_meta1,x_train_meta2,x_train_meta3,x_train_meta4],lbl_train,epochs=100,
                               validation_data=([x_test,x_test_meta1,x_test_meta2,x_test_meta3,x_test_meta4],lbl_test),
                               class_weight=class_weight,
                               #callbacks=[save_best_model],
                               shuffle=True,batch_size=100)

pred_test=CNN_embedding.predict([x_test,x_test_meta1,x_test_meta2,x_test_meta3,x_test_meta4])
lbl_pred=np.argmax(pred_test,axis=1).astype(int)
lbl_real=np.argmax(lbl_test,axis=1).astype(int)
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
#------------------------------------------------------------------------------

def BootstrapR2(pred_test,lbl_test, numboot=10000):
    lbl_pred=np.argmax(pred_test,axis=1).astype(int)
    lbl_real=np.argmax(lbl_test,axis=1).astype(int)
    classfi_report=classification_report(lbl_real, lbl_pred,output_dict=True)
    accuracy=accuracy_score(lbl_real, lbl_pred)
    precision=classfi_report['macro avg']['precision'] 
    recall= classfi_report['macro avg']['recall']    
    f1_score=classfi_report['macro avg']['f1-score']
    roc = roc_auc_score(lbl_test, pred_test,multi_class='ovr',average='weighted') 

    #---------------------------------
    n = len(lbl_test)
    roc1 = np.zeros((numboot, 1))
    f1_score1 = np.zeros((numboot, 1))
    accuracy1 = np.zeros((numboot, 1))
    precision1 = np.zeros((numboot, 1))
    recall1 = np.zeros((numboot, 1)) 
     
    for i in range(numboot):
        random_index1 = np.random.randint(0, lbl_pred.shape[0],size=len(lbl_test))
        lbl_test1=lbl_test[random_index1]
        pred_test1=pred_test[random_index1]

        lbl_pred1=np.argmax(pred_test1,axis=1).astype(int)
        lbl_real1=np.argmax(lbl_test1,axis=1).astype(int)

        classfi_report=classification_report(lbl_real1, lbl_pred1,output_dict=True)

        accuracy1[i]=accuracy_score(lbl_real1, lbl_pred1)
        precision1[i]=classfi_report['macro avg']['precision'] 
        recall1[i]= classfi_report['macro avg']['recall']    
        f1_score1[i]=classfi_report['macro avg']['f1-score']
        roc1[i] = roc_auc_score(lbl_test1, pred_test1,multi_class='ovr',average='weighted') 
#----------------------
    lbl_pred=np.argmax(pred_test,axis=1).astype(int)
    lbl_real=np.argmax(lbl_test,axis=1).astype(int)
    classfi_report=classification_report(lbl_real, lbl_pred,output_dict=True)
    accuracy=accuracy_score(lbl_real, lbl_pred)
    precision=classfi_report['macro avg']['precision'] 
    recall= classfi_report['macro avg']['recall']    
    f1_score=classfi_report['macro avg']['f1-score']
    roc = roc_auc_score(lbl_test, pred_test,multi_class='ovr',average='weighted') 


    lower,upper=np.quantile(roc1 - roc, [0.05, 0.95])
    print("AUC")
    print(round(roc,3),"", round(lower,3))
    print(round(roc,3),"+", round(upper,3))

    lower,upper=np.quantile(f1_score1 - f1_score, [0.05, 0.95])
    print("f1_score")
    print(round(f1_score,3),"", round(lower,3))
    print(round(f1_score,3),"+", round(upper,3))

    lower,upper=np.quantile(accuracy1 - accuracy, [0.05, 0.95])
    print("accuracy")
    print(round(accuracy,3),"", round(lower,3))
    print(round(accuracy,3),"+", round(upper,3))

    lower,upper=np.quantile(precision1 - precision, [0.05, 0.95])
    print("precision")
    print(round(precision,3),"", round(lower,3))
    print(round(precision,3),"+", round(upper,3))

    lower,upper=np.quantile(recall1 - recall, [0.05, 0.95])
    print("recall")
    print(round(recall,3),"", round(lower,3))
    print(round(recall,3),"+", round(upper,3))
        
    return roc1,f1_score1,accuracy1,precision1,recall1
roc1,f1_score1,accuracy1,precision1,recall1=BootstrapR2(pred_test,lbl_test, numboot=10000)

##########################################LSTM

def Create_CNN(data,nb_words,maxlen,embedding_dim,data_meta1,data_meta2,data_meta3,data_meta4):


  # meta1 input1
  input_meta1=keras.layers.Input(shape=data_meta1.shape[1:],dtype='float64',name="input_meta1")  
  layer_meta1=keras.layers.Conv1D(filters=64,
                                kernel_size=2,
                                activation='relu',
                                padding="same",
                                strides=(1),
                              name="cov_meta1.1")(input_meta1)
  layer_meta1 =keras.layers.Dropout(0.3)(layer_meta1)
  layer_meta1= keras.layers.LSTM(64,return_sequences=True)(layer_meta1)#return_sequences=False
    
  # meta2 input1
  input_meta2=keras.layers.Input(shape=data_meta2.shape[1:],dtype='float64',name="input_meta2")  
  layer_meta2=keras.layers.Conv1D(filters=64,
                                kernel_size=2,
                                activation='relu',
                                padding="same",
                                strides=(1),
                              name="cov_meta2.1")(input_meta2)
  layer_meta2 =keras.layers.Dropout(0.3)(layer_meta2)
  layer_meta2= keras.layers.LSTM(64,return_sequences=True)(layer_meta2)#return_sequences=False 
    
  # meta3 input1
  input_meta3=keras.layers.Input(shape=data_meta3.shape[1:],dtype='float64',name="input_meta3")  
  layer_meta3=keras.layers.Conv1D(filters=64,
                                kernel_size=2,
                                activation='relu',
                                padding="same",
                                strides=(1),
                              name="cov_meta3.1")(input_meta3)
  layer_meta3 =keras.layers.Dropout(0.3)(layer_meta3)
  layer_meta3= keras.layers.LSTM(64,return_sequences=True)(layer_meta3)#return_sequences=False
    
    
  # meta4 input1
  input_meta4=keras.layers.Input(shape=data_meta4.shape[1:],dtype='float64',name="input_meta4")  
  layer_meta4=keras.layers.Conv1D(filters=64,
                                kernel_size=2,
                                activation='relu',
                                padding="same",
                                strides=(1),
                              name="cov_meta4.1")(input_meta4)
  layer_meta4 =keras.layers.Dropout(0.3)(layer_meta4)
  layer_meta4= keras.layers.LSTM(64,return_sequences=True)(layer_meta4)#return_sequences=False


  # txt input
  word_input_cnn=keras.layers.Input(shape=(maxlen,),dtype='float64',name="txt_input_cnn")  
  em_layer_cnn=keras.layers.Embedding(input_dim=vocab_size, 
                                  output_dim=embedding_dim,input_length=maxlen,
                                  name="Embedd_cnn")(word_input_cnn)
  layer_data=keras.layers.Conv1D(filters=64,
                                 kernel_size=2,
                                 activation='relu',
                                 padding="same",
                                 strides=(2),
                                name="cov_1")(em_layer_cnn)
  layer_data=keras.layers.MaxPool1D(pool_size=(2),name="data_maxpool0")(layer_data)

  layer_data =keras.layers.BatchNormalization()(layer_data) 
  layer_data =keras.layers.Dropout(0.1)(layer_data)

  layer_data=keras.layers.Conv1D(filters=64,
                                 kernel_size=2,
                                 activation='relu',
                                 padding="same",
                                name="cov_2")(layer_data)
  layer_data=keras.layers.MaxPool1D(pool_size=(2),name="data_maxpool1")(layer_data)

  layer_data =keras.layers.BatchNormalization()(layer_data)                              
  layer_data =keras.layers.Dropout(0.1)(layer_data)
  layer_data=keras.layers.Conv1D(filters=64,
                                 kernel_size=2,
                                 strides=(2),
                                 activation='relu',
                                 padding="same",
                                name="cov_4")(layer_data)
  layer_data =keras.layers.Dropout(0.3)(layer_data)
  layer_data= keras.layers.LSTM(64,return_sequences=True)(layer_data)#return_sequences=False
  #layer_data=keras.layers.GlobalAveragePooling1D(name="data_maxpool2")(layer_data)



  # hybrid
  layer_last1 =keras.layers.Concatenate(axis=1)([layer_meta1,layer_meta2,layer_meta3,layer_meta4]) 
  layer_last = MultiHeadAttention(num_heads=1, key_dim=5, dropout=0.5, name = "MHSA")(layer_last1,layer_data)
  layer_last=keras.layers.Flatten()(layer_last)

  
  layer_last =keras.layers.Dense(64,activation="relu",
                                 name="dens_2")(layer_last)
  layer_last =keras.layers.Dropout(0.5)(layer_last)
  output= keras.layers.Dense(n_class, activation='softmax', name='out')(layer_last)
  model = Model(inputs=[word_input_cnn,input_meta1,input_meta2,input_meta3,input_meta4],
                       outputs=output)
  opti=keras.optimizers.Adam(learning_rate=0.00001)
  model.compile(loss='categorical_crossentropy',
                       optimizer=opti,
                       metrics=[METRICS])
  return model


# In[101]:


CNN_embedding=Create_CNN(data=data_seq_pad,
                      nb_words=MAX_NB_WORDS,
                      maxlen=maxlen,
                      embedding_dim=embedding_dim,
                      data_meta1=x_train_meta_re1,
                      data_meta2=x_train_meta_re2,
                      data_meta3=x_train_meta_re3,
                      data_meta4=x_train_meta_re4)

hist_CNN_embedding=CNN_embedding.fit([x_train,x_train_meta1,x_train_meta2,x_train_meta3,x_train_meta4],lbl_train,epochs=100,
                               validation_data=([x_test,x_test_meta1,x_test_meta2,x_test_meta3,x_test_meta4],lbl_test),
                               class_weight=class_weight,
                               #callbacks=[save_best_model],
                               shuffle=True,batch_size=100)

pred_test=CNN_embedding.predict([x_test,x_test_meta1,x_test_meta2,x_test_meta3,x_test_meta4])
lbl_pred=np.argmax(pred_test,axis=1).astype(int)
lbl_real=np.argmax(lbl_test,axis=1).astype(int)
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
#------------------------------------------------------------------------------

def BootstrapR2(pred_test,lbl_test, numboot=10000):
    lbl_pred=np.argmax(pred_test,axis=1).astype(int)
    lbl_real=np.argmax(lbl_test,axis=1).astype(int)
    classfi_report=classification_report(lbl_real, lbl_pred,output_dict=True)
    accuracy=accuracy_score(lbl_real, lbl_pred)
    precision=classfi_report['macro avg']['precision'] 
    recall= classfi_report['macro avg']['recall']    
    f1_score=classfi_report['macro avg']['f1-score']
    roc = roc_auc_score(lbl_test, pred_test,multi_class='ovr',average='weighted') 

    #---------------------------------
    n = len(lbl_test)
    roc1 = np.zeros((numboot, 1))
    f1_score1 = np.zeros((numboot, 1))
    accuracy1 = np.zeros((numboot, 1))
    precision1 = np.zeros((numboot, 1))
    recall1 = np.zeros((numboot, 1)) 
     
    for i in range(numboot):
        random_index1 = np.random.randint(0, lbl_pred.shape[0],size=len(lbl_test))
        lbl_test1=lbl_test[random_index1]
        pred_test1=pred_test[random_index1]

        lbl_pred1=np.argmax(pred_test1,axis=1).astype(int)
        lbl_real1=np.argmax(lbl_test1,axis=1).astype(int)

        classfi_report=classification_report(lbl_real1, lbl_pred1,output_dict=True)

        accuracy1[i]=accuracy_score(lbl_real1, lbl_pred1)
        precision1[i]=classfi_report['macro avg']['precision'] 
        recall1[i]= classfi_report['macro avg']['recall']    
        f1_score1[i]=classfi_report['macro avg']['f1-score']
        roc1[i] = roc_auc_score(lbl_test1, pred_test1,multi_class='ovr',average='weighted') 
#----------------------
    lbl_pred=np.argmax(pred_test,axis=1).astype(int)
    lbl_real=np.argmax(lbl_test,axis=1).astype(int)
    classfi_report=classification_report(lbl_real, lbl_pred,output_dict=True)
    accuracy=accuracy_score(lbl_real, lbl_pred)
    precision=classfi_report['macro avg']['precision'] 
    recall= classfi_report['macro avg']['recall']    
    f1_score=classfi_report['macro avg']['f1-score']
    roc = roc_auc_score(lbl_test, pred_test,multi_class='ovr',average='weighted') 


    lower,upper=np.quantile(roc1 - roc, [0.05, 0.95])
    print("AUC")
    print(round(roc,3),"", round(lower,3))
    print(round(roc,3),"+", round(upper,3))

    lower,upper=np.quantile(f1_score1 - f1_score, [0.05, 0.95])
    print("f1_score")
    print(round(f1_score,3),"", round(lower,3))
    print(round(f1_score,3),"+", round(upper,3))

    lower,upper=np.quantile(accuracy1 - accuracy, [0.05, 0.95])
    print("accuracy")
    print(round(accuracy,3),"", round(lower,3))
    print(round(accuracy,3),"+", round(upper,3))

    lower,upper=np.quantile(precision1 - precision, [0.05, 0.95])
    print("precision")
    print(round(precision,3),"", round(lower,3))
    print(round(precision,3),"+", round(upper,3))

    lower,upper=np.quantile(recall1 - recall, [0.05, 0.95])
    print("recall")
    print(round(recall,3),"", round(lower,3))
    print(round(recall,3),"+", round(upper,3))
        
    return roc1,f1_score1,accuracy1,precision1,recall1
roc1,f1_score1,accuracy1,precision1,recall1=BootstrapR2(pred_test,lbl_test, numboot=10000)
###############################################GRU

def Create_CNN(data,nb_words,maxlen,embedding_dim,data_meta1,data_meta2,data_meta3,data_meta4):


 # meta1 input1
  input_meta1=keras.layers.Input(shape=data_meta1.shape[1:],dtype='float64',name="input_meta1")  
  layer_meta1=keras.layers.Conv1D(filters=64,
                                kernel_size=2,
                                activation='relu',
                                padding="same",
                                strides=(1),
                              name="cov_meta1.1")(input_meta1)
  layer_meta1 =keras.layers.Dropout(0.1)(layer_meta1)
  layer_meta1=GRU(128, return_sequences=True)(layer_meta1) 
    
  # meta2 input1
  input_meta2=keras.layers.Input(shape=data_meta2.shape[1:],dtype='float64',name="input_meta2")  
  layer_meta2=keras.layers.Conv1D(filters=64,
                                kernel_size=2,
                                activation='relu',
                                padding="same",
                                strides=(1),
                              name="cov_meta2.1")(input_meta2)
  layer_meta2 =keras.layers.Dropout(0.1)(layer_meta2)
  layer_meta2=GRU(128, return_sequences=True)(layer_meta2) 
    
  # meta3 input1
  input_meta3=keras.layers.Input(shape=data_meta3.shape[1:],dtype='float64',name="input_meta3")  
  layer_meta3=keras.layers.Conv1D(filters=64,
                                kernel_size=2,
                                activation='relu',
                                padding="same",
                                strides=(1),
                              name="cov_meta3.1")(input_meta3)
  layer_meta3 =keras.layers.Dropout(0.1)(layer_meta3)
  layer_meta3=GRU(128, return_sequences=True)(layer_meta3) 
    
    
  # meta4 input1
  input_meta4=keras.layers.Input(shape=data_meta4.shape[1:],dtype='float64',name="input_meta4")  
  layer_meta4=keras.layers.Conv1D(filters=64,
                                kernel_size=2,
                                activation='relu',
                                padding="same",
                                strides=(1),
                              name="cov_meta4.1")(input_meta4)
  layer_meta4 =keras.layers.Dropout(0.1)(layer_meta4)
  layer_meta4=GRU(128, return_sequences=True)(layer_meta4) 

  # txt input
  word_input_cnn=keras.layers.Input(shape=(maxlen,),dtype='float64',name="txt_input_cnn")  
  em_layer_cnn=keras.layers.Embedding(input_dim=vocab_size, 
                                  output_dim=embedding_dim,input_length=maxlen,
                                  name="Embedd_cnn")(word_input_cnn)
  layer_data=keras.layers.Conv1D(filters=64,
                                 kernel_size=2,
                                 activation='relu',
                                 padding="same",
                                 strides=(2),
                                name="cov_1")(em_layer_cnn)
  layer_data=keras.layers.MaxPool1D(pool_size=(2),name="data_maxpool0")(layer_data)

  layer_data =keras.layers.BatchNormalization()(layer_data) 
  layer_data =keras.layers.Dropout(0.1)(layer_data)

  layer_data=keras.layers.Conv1D(filters=64,
                                 kernel_size=2,
                                 activation='relu',
                                 padding="same",
                                name="cov_2")(layer_data)
  layer_data=keras.layers.MaxPool1D(pool_size=(2),name="data_maxpool1")(layer_data)

  layer_data =keras.layers.BatchNormalization()(layer_data)                              
  layer_data =keras.layers.Dropout(0.1)(layer_data)
  layer_data=GRU(128, return_sequences=True)(layer_data)
#  layer_data=keras.layers.GlobalAveragePooling1D(name="data_maxpool2")(layer_data)



  # hybrid
  layer_last1 =keras.layers.Concatenate(axis=1)([layer_meta1,layer_meta2,layer_meta3,layer_meta4]) 
  layer_last = MultiHeadAttention(num_heads=1, key_dim=5, dropout=0.5, name = "MHSA")(layer_last1,layer_data)
  layer_last=keras.layers.Flatten()(layer_last)

  
  layer_last =keras.layers.Dense(64,activation="relu",
                                 name="dens_2")(layer_last)
  layer_last =keras.layers.Dropout(0.5)(layer_last)
  output= keras.layers.Dense(n_class, activation='softmax', name='out')(layer_last)
  model = Model(inputs=[word_input_cnn,input_meta1,input_meta2,input_meta3,input_meta4],
                       outputs=output)
  opti=keras.optimizers.Adam(learning_rate=0.00001)
  model.compile(loss='categorical_crossentropy',
                       optimizer=opti,
                       metrics=[METRICS])
  return model


# In[101]:


CNN_embedding=Create_CNN(data=data_seq_pad,
                      nb_words=MAX_NB_WORDS,
                      maxlen=maxlen,
                      embedding_dim=embedding_dim,
                      data_meta1=x_train_meta_re1,
                      data_meta2=x_train_meta_re2,
                      data_meta3=x_train_meta_re3,
                      data_meta4=x_train_meta_re4)

hist_CNN_embedding=CNN_embedding.fit([x_train,x_train_meta1,x_train_meta2,x_train_meta3,x_train_meta4],lbl_train,epochs=100,
                               validation_data=([x_test,x_test_meta1,x_test_meta2,x_test_meta3,x_test_meta4],lbl_test),
                               class_weight=class_weight,
                               #callbacks=[save_best_model],
                               shuffle=True,batch_size=100)

pred_test=CNN_embedding.predict([x_test,x_test_meta1,x_test_meta2,x_test_meta3,x_test_meta4])
lbl_pred=np.argmax(pred_test,axis=1).astype(int)
lbl_real=np.argmax(lbl_test,axis=1).astype(int)
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
#------------------------------------------------------------------------------

def BootstrapR2(pred_test,lbl_test, numboot=10000):
    lbl_pred=np.argmax(pred_test,axis=1).astype(int)
    lbl_real=np.argmax(lbl_test,axis=1).astype(int)
    classfi_report=classification_report(lbl_real, lbl_pred,output_dict=True)
    accuracy=accuracy_score(lbl_real, lbl_pred)
    precision=classfi_report['macro avg']['precision'] 
    recall= classfi_report['macro avg']['recall']    
    f1_score=classfi_report['macro avg']['f1-score']
    roc = roc_auc_score(lbl_test, pred_test,multi_class='ovr',average='weighted') 

    #---------------------------------
    n = len(lbl_test)
    roc1 = np.zeros((numboot, 1))
    f1_score1 = np.zeros((numboot, 1))
    accuracy1 = np.zeros((numboot, 1))
    precision1 = np.zeros((numboot, 1))
    recall1 = np.zeros((numboot, 1)) 
     
    for i in range(numboot):
        random_index1 = np.random.randint(0, lbl_pred.shape[0],size=len(lbl_test))
        lbl_test1=lbl_test[random_index1]
        pred_test1=pred_test[random_index1]

        lbl_pred1=np.argmax(pred_test1,axis=1).astype(int)
        lbl_real1=np.argmax(lbl_test1,axis=1).astype(int)

        classfi_report=classification_report(lbl_real1, lbl_pred1,output_dict=True)

        accuracy1[i]=accuracy_score(lbl_real1, lbl_pred1)
        precision1[i]=classfi_report['macro avg']['precision'] 
        recall1[i]= classfi_report['macro avg']['recall']    
        f1_score1[i]=classfi_report['macro avg']['f1-score']
        roc1[i] = roc_auc_score(lbl_test1, pred_test1,multi_class='ovr',average='weighted') 
#----------------------
    lbl_pred=np.argmax(pred_test,axis=1).astype(int)
    lbl_real=np.argmax(lbl_test,axis=1).astype(int)
    classfi_report=classification_report(lbl_real, lbl_pred,output_dict=True)
    accuracy=accuracy_score(lbl_real, lbl_pred)
    precision=classfi_report['macro avg']['precision'] 
    recall= classfi_report['macro avg']['recall']    
    f1_score=classfi_report['macro avg']['f1-score']
    roc = roc_auc_score(lbl_test, pred_test,multi_class='ovr',average='weighted') 


    lower,upper=np.quantile(roc1 - roc, [0.05, 0.95])
    print("AUC")
    print(round(roc,3),"", round(lower,3))
    print(round(roc,3),"+", round(upper,3))

    lower,upper=np.quantile(f1_score1 - f1_score, [0.05, 0.95])
    print("f1_score")
    print(round(f1_score,3),"", round(lower,3))
    print(round(f1_score,3),"+", round(upper,3))

    lower,upper=np.quantile(accuracy1 - accuracy, [0.05, 0.95])
    print("accuracy")
    print(round(accuracy,3),"", round(lower,3))
    print(round(accuracy,3),"+", round(upper,3))

    lower,upper=np.quantile(precision1 - precision, [0.05, 0.95])
    print("precision")
    print(round(precision,3),"", round(lower,3))
    print(round(precision,3),"+", round(upper,3))

    lower,upper=np.quantile(recall1 - recall, [0.05, 0.95])
    print("recall")
    print(round(recall,3),"", round(lower,3))
    print(round(recall,3),"+", round(upper,3))
        
    return roc1,f1_score1,accuracy1,precision1,recall1
roc1,f1_score1,accuracy1,precision1,recall1=BootstrapR2(pred_test,lbl_test, numboot=10000)
#######################################BERT

maxlen_bert=500
#from bert import bert_tokenization
from bert import tokenization
import tensorflow_hub as hub
with strategy.scope():
   # BertTokenizer = bert_tokenization.FullTokenizer
    BertTokenizer = tokenization.FullTokenizer

with strategy.scope():
    bert_layer = hub.KerasLayer("EXT/DDD/ce53fe6769d2ac3a260e92555120c54e1aecbea6",trainable=False)

    vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
    to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
    tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

    
with strategy.scope():
    def bert_encode(texts, tokenizer, max_len=512):
        all_tokens = []
        all_masks = []
        all_segments = []

        for text in texts:
            text = tokenizer.tokenize(text)

            text = text[:max_len-2]
            input_sequence = ["[CLS]"] + text + ["[SEP]"]
            pad_len = max_len - len(input_sequence)

            tokens = tokenizer.convert_tokens_to_ids(input_sequence)
            tokens += [0] * pad_len
            pad_masks = [1] * len(input_sequence) + [0] * pad_len
            segment_ids = [0] * max_len

            all_tokens.append(tokens)
            all_masks.append(pad_masks)
            all_segments.append(segment_ids)

        return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

    import sys
from absl import flags
sys.argv=['preserve_unused_tokens=False']
flags.FLAGS(sys.argv)
with strategy.scope():
    train_input_bert = bert_encode(x_train_txt, tokenizer, max_len=maxlen_bert)
    test_input_bert = bert_encode(x_test_txt, tokenizer, max_len=maxlen_bert)


    
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

def create_beart_model(bert_layer, max_len=512,data_meta1=[],data_meta2=[],data_meta3=[],data_meta4=[]):


   # meta1 input1
    input_meta1=keras.layers.Input(shape=data_meta1.shape[1:],dtype='float64',name="input_meta1")  
    layer_meta1=keras.layers.Conv1D(filters=64,
                                   kernel_size=2,
                                   activation='relu',
                                   padding="same",
                                   strides=(1),
                                   name="cov_meta1.1")(input_meta1)
    layer_meta1=keras.layers.MaxPool1D(pool_size=(2),name="meta1_maxpool1")(layer_meta1)
    layer_meta1 = MultiHeadAttention(num_heads=1, key_dim=5, dropout=0.5, name = "MHSA1")(layer_meta1,layer_meta1)
    #layer_meta1=keras.layers.GlobalAveragePooling1D(name="avg_meta1")(layer_meta1)   
  # meta2 input1
    input_meta2=keras.layers.Input(shape=data_meta2.shape[1:],dtype='float64',name="input_meta2")  
    layer_meta2=keras.layers.Conv1D(filters=64,
                                   kernel_size=2,
                                   activation='relu',
                                   padding="same",
                                   strides=(1),
                                   name="cov_meta2.1")(input_meta2)
    layer_meta2=keras.layers.MaxPool1D(pool_size=(2),name="meta2_maxpool1")(layer_meta2)
    layer_meta2 = MultiHeadAttention(num_heads=1, key_dim=5, dropout=0.5, name = "MHSA2")(layer_meta2,layer_meta2)
    #layer_meta2=keras.layers.GlobalAveragePooling1D(name="avg_meta2")(layer_meta2)       
  # meta3 input1
    input_meta3=keras.layers.Input(shape=data_meta3.shape[1:],dtype='float64',name="input_meta3")  
    layer_meta3=keras.layers.Conv1D(filters=64,
                                   kernel_size=2,
                                   activation='relu',
                                   padding="same",
                                   strides=(1),
                                   name="cov_meta3.1")(input_meta3)
    layer_meta3=keras.layers.MaxPool1D(pool_size=(2),name="meta3_maxpool1")(layer_meta3)
    layer_meta3 = MultiHeadAttention(num_heads=1, key_dim=5, dropout=0.5, name = "MHSA3")(layer_meta3,layer_meta3)
    #layer_meta3=keras.layers.GlobalAveragePooling1D(name="avg_meta3")(layer_meta3)      
  # meta4 input1
    input_meta4=keras.layers.Input(shape=data_meta4.shape[1:],dtype='float64',name="input_meta4")  
    layer_meta4=keras.layers.Conv1D(filters=64,
                                   kernel_size=2,
                                   activation='relu',
                                   padding="same",
                                   strides=(1),
                                   name="cov_meta4.1")(input_meta4)
    layer_meta4=keras.layers.MaxPool1D(pool_size=(2),name="meta4_maxpool1")(layer_meta4)
    layer_meta4 = MultiHeadAttention(num_heads=1, key_dim=5, dropout=0.5, name = "MHSA4")(layer_meta4,layer_meta4)
    #layer_meta4=keras.layers.GlobalAveragePooling1D(name="avg_meta4")(layer_meta4)


# txt input

    input_word_ids = keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = keras.layers.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    layer_bert = sequence_output[:, :, :]

    #layer_bert=keras.layers.GlobalAveragePooling1D(name="avg1")(layer_bert)

  # hybrid
    layer_last1 =keras.layers.Concatenate(axis=1)([layer_meta1,layer_meta2,layer_meta3,layer_meta4]) 
    layer_last = MultiHeadAttention(num_heads=5, key_dim=5, dropout=0.5, name = "MHSA")(layer_last1,layer_bert)
    layer_last=keras.layers.Flatten()(layer_last) 
    layer_last =keras.layers.Dense(64,activation="relu",name="dens_2")(layer_last)
    layer_last =keras.layers.Dropout(0.5)(layer_last)
    output= keras.layers.Dense(n_class, activation='softmax', name='out')(layer_last)
    bert_model = keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids,input_meta1,input_meta2,input_meta3,input_meta4], outputs=output)
    bert_model.compile(loss='categorical_crossentropy',
                       optimizer=keras.optimizers.Adam(learning_rate=0.00001),
                       metrics=["accuracy"])
    return bert_model


# In[101]:


model_bert = create_beart_model(bert_layer,               
                                max_len=maxlen_bert, 
                                data_meta1=x_train_meta_re1,
                                data_meta2=x_train_meta_re2,
                                data_meta3=x_train_meta_re3, 
                                data_meta4=x_train_meta_re4)


train_bert_new1=train_input_bert+(x_train_meta_re1,)+(x_train_meta_re2,)+(x_train_meta_re3,)+(x_train_meta_re4,)
test_bert_new1=test_input_bert+(x_test_meta_re1,)+(x_test_meta_re2,)+(x_test_meta_re3,)+(x_test_meta_re4,)

history=model_bert.fit(train_bert_new1,lbl_train,epochs=300,
                               validation_data=(test_bert_new1,lbl_test),
                               class_weight=class_weight,
                               #callbacks=[save_best_model],
                               shuffle=True,batch_size=100)

pred_test=model_bert.predict(test_bert_new1)

lbl_pred=np.argmax(pred_test,axis=1).astype(int)
lbl_real=np.argmax(lbl_test,axis=1).astype(int)
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
#------------------------------------------------------------------------------

def BootstrapR2(pred_test,lbl_test, numboot=10000):
    lbl_pred=np.argmax(pred_test,axis=1).astype(int)
    lbl_real=np.argmax(lbl_test,axis=1).astype(int)
    classfi_report=classification_report(lbl_real, lbl_pred,output_dict=True)
    accuracy=accuracy_score(lbl_real, lbl_pred)
    precision=classfi_report['macro avg']['precision'] 
    recall= classfi_report['macro avg']['recall']    
    f1_score=classfi_report['macro avg']['f1-score']
    roc = roc_auc_score(lbl_test, pred_test,multi_class='ovr',average='weighted') 

    #---------------------------------
    n = len(lbl_test)
    roc1 = np.zeros((numboot, 1))
    f1_score1 = np.zeros((numboot, 1))
    accuracy1 = np.zeros((numboot, 1))
    precision1 = np.zeros((numboot, 1))
    recall1 = np.zeros((numboot, 1)) 
     
    for i in range(numboot):
        random_index1 = np.random.randint(0, lbl_pred.shape[0],size=len(lbl_test))
        lbl_test1=lbl_test[random_index1]
        pred_test1=pred_test[random_index1]

        lbl_pred1=np.argmax(pred_test1,axis=1).astype(int)
        lbl_real1=np.argmax(lbl_test1,axis=1).astype(int)

        classfi_report=classification_report(lbl_real1, lbl_pred1,output_dict=True)

        accuracy1[i]=accuracy_score(lbl_real1, lbl_pred1)
        precision1[i]=classfi_report['macro avg']['precision'] 
        recall1[i]= classfi_report['macro avg']['recall']    
        f1_score1[i]=classfi_report['macro avg']['f1-score']
        roc1[i] = roc_auc_score(lbl_test1, pred_test1,multi_class='ovr',average='weighted') 
#----------------------
    lbl_pred=np.argmax(pred_test,axis=1).astype(int)
    lbl_real=np.argmax(lbl_test,axis=1).astype(int)
    classfi_report=classification_report(lbl_real, lbl_pred,output_dict=True)
    accuracy=accuracy_score(lbl_real, lbl_pred)
    precision=classfi_report['macro avg']['precision'] 
    recall= classfi_report['macro avg']['recall']    
    f1_score=classfi_report['macro avg']['f1-score']
    roc = roc_auc_score(lbl_test, pred_test,multi_class='ovr',average='weighted') 


    lower,upper=np.quantile(roc1 - roc, [0.05, 0.95])
    print("AUC")
    print(round(roc,3),"", round(lower,3))
    print(round(roc,3),"+", round(upper,3))

    lower,upper=np.quantile(f1_score1 - f1_score, [0.05, 0.95])
    print("f1_score")
    print(round(f1_score,3),"", round(lower,3))
    print(round(f1_score,3),"+", round(upper,3))

    lower,upper=np.quantile(accuracy1 - accuracy, [0.05, 0.95])
    print("accuracy")
    print(round(accuracy,3),"", round(lower,3))
    print(round(accuracy,3),"+", round(upper,3))

    lower,upper=np.quantile(precision1 - precision, [0.05, 0.95])
    print("precision")
    print(round(precision,3),"", round(lower,3))
    print(round(precision,3),"+", round(upper,3))

    lower,upper=np.quantile(recall1 - recall, [0.05, 0.95])
    print("recall")
    print(round(recall,3),"", round(lower,3))
    print(round(recall,3),"+", round(upper,3))
        
    return roc1,f1_score1,accuracy1,precision1,recall1
roc1,f1_score1,accuracy1,precision1,recall1=BootstrapR2(pred_test,lbl_test, numboot=10000)